IVF INDEXING USING FAISS

In [ ]:
!pip install -qq faiss-cpu  langchain-community


In [ ]:
!pip install -qq langchain_huggingface

In [ ]:
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter

model = SentenceTransformer('all-MiniLM-L6-V2')

document = """
The Taj Mahal is a white marble mausoleum located in Agra, India.
It was commissioned in 1632 by the Mughal emperor Shah Jahan to house the tomb of his favorite wife, Mumtaz Mahal.
The Taj Mahal is widely recognized as the jewel of Muslim art in India and one of the universally admired masterpieces of world heritage.
It attracts millions of tourists every year.
"""


splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 15

)

chunks = splitter.split_text(document)

embedding = model.encode(chunks,convert_to_numpy=True)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

next step is IVF index we going train[link text](https://)

In [ ]:
import faiss
import numpy as np

quantizer =  faiss.IndexFlatL2(embedding.shape[1])

# Step 2: Create IVF index (use nlist = 2 for this small example)
ivf_index =  faiss.IndexIVFFlat(quantizer,embedding.shape[1],2)

ivf_index.train(embedding)

ivf_index.add(embedding)



In [ ]:
print("vector stored in IVF index:",ivf_index.ntotal)

vector stored in IVF index: 6


✅ Step 2: Wrap the IVF FAISS Index in LangChain

In [ ]:
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain_huggingface import HuggingFaceEmbeddings

docs = [Document(page_content=chunk) for chunk in chunks]

langchain_huggingface =  HuggingFaceEmbeddings(model_name='all-MiniLM-L6-V2')

vectorstore =  FAISS.from_documents(docs,embedding= langchain_huggingface )

retriever = vectorstore.as_retriever()

In [ ]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA

# Step 1: Load HuggingFace LLM pipeline
hf_pipeline = pipeline("text2text-generation", model="google/flan-t5-base", max_new_tokens=100)

# Step 2: Wrap in LangChain-compatible LLM
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# Step 3: Create RetrievalQA (RAG) chain
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# Step 4: Ask a question
query = "Who built the Taj Mahal?"
answer = qa_chain.run(query)
print("Answer:", answer)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu
/tmp/ipython-input-28-903360703.py:9: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)
/tmp/ipython-input-28-903360703.py:16: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain.run(query)


Answer: Mughal emperor Shah Jahan


In [ ]:
# Step 4: Ask a question
query = """what is taj mahal"""
answer = qa_chain.run(query)
print("Answer:", answer)

Answer: The Taj Mahal is widely recognized as the jewel of Muslim art in India and one of the universally The Taj Mahal is a white marble mausoleum located in Agra, India. his favorite wife, Mumtaz Mahal. It was commissioned in 1632 by the Mughal emperor Shah Jahan to house the tomb of his favorite
